In [1]:
%env KERAS_BACKEND=tensorflow
%matplotlib inline

env: KERAS_BACKEND=tensorflow


In [2]:
import pandas as pd
import numpy as np
import glob
import os

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.losses import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


## Load data

In [3]:
# load all csv files of North-point site. Concatenate them..
data_src = "../data"
data_dst = "../output"
all_files = glob.glob(os.path.join(data_src, "np", "*.csv"))
df = pd.concat([pd.read_csv(f, low_memory=False) for f in all_files], ignore_index=True)

# minor changes
df = df.rename(columns={"Time Stamp": "timestamp"})
df = df.replace("\\N", np.nan)

# update data types. object is taken as default
dtypes = dict([(col, np.float64) for col in df.columns])
dtypes["timestamp"] = "datetime64[ns]"
df = df.astype(dtypes)

# change the index to timestamp.
df.index = df.timestamp

## Data sanitizing

In [4]:
# Sample time series. Let's select a period and model for that.. 
ts = df["2017-01-01": "2017-01"]

**TODO:** Split the time series into chunks. Each chunk breaks when the field value is null. Train all the chunks separately.

In [5]:
# Preprocessing: Interpolate null values in the `fields`
# Note: This does not eliminate all null values if their continuous sequence > window size.
fields = ["loadsys", "wetbulb", "ct1kw", "ct2kw", "ct3kw", "cwshdr"]
rolling_avg = ts[fields].rolling(10, min_periods=1).mean()

# the time series after some sanitization
ts = ts[fields].fillna(rolling_avg).fillna(method="ffill").dropna()

In [6]:
# Normalize this data..
scaler = MinMaxScaler(feature_range=(0,1))
normalized_ts = scaler.fit_transform(ts)
normalized_ts = pd.DataFrame(normalized_ts, columns=fields, index=ts.index.values)

In [7]:
# split the dataframe for training and validation
# dataframe  := Pandas dataframe
# ratio := Float, training:validation
def train_validation_split(dataframe, ratio):
    size = len(dataframe)
    train_df = dataframe.iloc[0:int(size*ratio)]
    validation_df = dataframe.iloc[int(size*ratio):]
    return train_df, validation_df

train_df, validation_df = train_validation_split(normalized_ts, 0.7)

In [11]:
# prepare feature vectors. the hypothesis is that
# y(t) can be determined using x1(k), x2(k), x3(k).... for all k = {t-1, t-2, t-3, ... t-N}, where 0 <= N <= t-1
def prepare_features(dataframe, target_field, N=1):
    x, y = [], []
    for i in range(len(dataframe)-N-1):
        x.append(dataframe.values[i:i+N])
        y.append(dataframe[target_field].values[i+N])
    x = np.array(x)
    y = np.array(y)
    return x, y

train_x, train_y = prepare_features(train_df, "cwshdr", N=5)
validation_x, validation_y = prepare_features(validation_df, "cwshdr", N=5)

# reshape data. combine all data points corresponding to a y(t)
reshape = lambda a: a.reshape((a.shape[0], a.shape[1] * a.shape[2]))
train_x = reshape(train_x)
validation_x = reshape(validation_x)
train_x.shape, train_y.shape

((30677, 30), (30677,))

## Prepare model

In [12]:
model = Sequential([
    LSTM(50, input_shape=(None, train_x.shape[1]), return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    LSTM(50, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    Dense(1)
])

model.compile(loss=mean_squared_error, optimizer="adam", metrics=["accuracy"])

## Train model

In [13]:
# This is how keras wants!!
reshape_x = lambda a: a.reshape((a.shape[0], 1, a.shape[1]))
reshape_y = lambda a: a.reshape((a.shape[0], 1, 1))

history = model.fit(
    x=reshape_x(train_x),
    y=reshape_y(train_y),
    validation_data=(reshape_x(validation_x), reshape_y(validation_y)),
    epochs=100)

Train on 30677 samples, validate on 13144 samples
Epoch 1/100
30677/30677 [==============================] - 27s - loss: 0.0212 - acc: 0.0109 - val_loss: 0.0299 - val_acc: 0.0104
Epoch 2/100
30677/30677 [==============================] - 24s - loss: 0.0024 - acc: 0.0133 - val_loss: 0.0102 - val_acc: 0.0923
Epoch 3/100
30677/30677 [==============================] - 26s - loss: 0.0015 - acc: 0.0140 - val_loss: 0.0042 - val_acc: 0.0924
Epoch 4/100
30677/30677 [==============================] - 22s - loss: 0.0013 - acc: 0.0140 - val_loss: 0.0019 - val_acc: 0.0924
Epoch 5/100
30677/30677 [==============================] - 24s - loss: 0.0012 - acc: 0.0140 - val_loss: 0.0019 - val_acc: 0.0924
Epoch 6/100
30677/30677 [==============================] - 28s - loss: 0.0011 - acc: 0.0140 - val_loss: 0.0020 - val_acc: 0.0924
Epoch 7/100
30677/30677 [==============================] - 25s - loss: 0.0011 - acc: 0.0140 - val_loss: 0.0021 - val_acc: 0.0924
Epoch 8/100
30677/30677 [======================

30677/30677 [==============================] - 25s - loss: 3.3053e-04 - acc: 0.0141 - val_loss: 0.0064 - val_acc: 0.0927
Epoch 63/100
30677/30677 [==============================] - 27s - loss: 3.5898e-04 - acc: 0.0141 - val_loss: 0.0055 - val_acc: 0.0927
Epoch 64/100
30677/30677 [==============================] - 30s - loss: 3.3607e-04 - acc: 0.0141 - val_loss: 0.0058 - val_acc: 0.0927
Epoch 65/100
30677/30677 [==============================] - 28s - loss: 3.5037e-04 - acc: 0.0141 - val_loss: 0.0052 - val_acc: 0.0927
Epoch 66/100
30677/30677 [==============================] - 32s - loss: 3.4197e-04 - acc: 0.0141 - val_loss: 0.0056 - val_acc: 0.0927
Epoch 67/100
30677/30677 [==============================] - 32s - loss: 3.6683e-04 - acc: 0.0141 - val_loss: 0.0051 - val_acc: 0.0926
Epoch 68/100
30677/30677 [==============================] - 32s - loss: 3.4553e-04 - acc: 0.0141 - val_loss: 0.0048 - val_acc: 0.0927
Epoch 69/100
30677/30677 [==============================] - 32s - loss: 3.6

## Metrics

In [ ]:
loss_list = np.array([history.history["loss"][0] for h in hist_list])
ax = plt.subplot(111)
ax.plot(loss_list)
ax.set_ylim(0.0007, 0.0015)
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")

In [ ]:
model_file = os.path.join(data_dst, "lstm2.h5")
model.save(model_file)

model_arch_file = os.path.join(data_src, "lstm2.json")
with open(model_arch_file, "wb") as fp:
    fp.write(model.to_json().encode())

## Forecast

In [ ]:
df_plot = pd.DataFrame({"original": validation_y})
ax = df_plot.plot(figsize=(36,6))